In [1]:
pip install simple_salesforce

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 18.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [simple_salesforce]zeep]ography]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import datetime as dt
import pandas as pd
import os
from simple_salesforce import Salesforce
from simple_salesforce.exceptions import SalesforceGeneralError

# ---- CONNECT (sandbox; session id is fine for local testing) ----
sf = Salesforce(
    instance_url="https://salesassist--full.sandbox.my.salesforce.com",
    # session_id="",
    version="61.0"
)
print("Connected to:", sf.sf_instance)

def run_query(soql: str):
    try:
        return sf.query_all(soql).get("records", [])
    except SalesforceGeneralError as e:
        raise RuntimeError(f"SOQL failed: {e.content}") from e

def to_df(records):
    if not records: return pd.DataFrame()
    for r in records: r.pop("attributes", None)
    return pd.DataFrame.from_records(records)

# ---- MAIN SOQL (30d window; NCO Web Sampling only) ----
soql_orders = """
SELECT Id, Name,
       ADC_Status__c,
       ADC_Order_Type__c,
       CreatedDate,
       ADC_Order_Approved_Date__c,
       ADC_Reason__c,
       ADC_Account_Name__c
FROM ADC_Compliance_Order__c
WHERE CreatedDate = LAST_N_DAYS:30
AND ADC_Order_Type__c = 'NCO Web Sampling'
"""

orders_df = to_df(run_query(soql_orders))

# Optional: show distinct order types in the 30d window to verify values
types_probe = to_df(run_query("""
SELECT ADC_Order_Type__c
FROM ADC_Compliance_Order__c
WHERE CreatedDate = LAST_N_DAYS:30
"""))
if not types_probe.empty:
    print("Order types in last 30d:", sorted(types_probe["ADC_Order_Type__c"].dropna().unique().tolist()))

if orders_df.empty:
    print("No orders found for LAST_N_DAYS:30 with ADC_Order_Type__c = 'NCO Web Sampling'.")
else:
    # ---- Cast dates ----
    orders_df["CreatedDate"] = pd.to_datetime(orders_df["CreatedDate"], utc=True)
    if "ADC_Order_Approved_Date__c" in orders_df.columns:
        orders_df["ADC_Order_Approved_Date__c"] = pd.to_datetime(
            orders_df["ADC_Order_Approved_Date__c"], errors="coerce"
        )

    # ---- KPIs ----
    now = pd.Timestamp.utcnow()
    last7 = now - pd.Timedelta(days=7)

    # Status counts (30d + 7d)
    kpi_30 = orders_df["ADC_Status__c"].value_counts().rename_axis("Status").reset_index(name="Count_30d")
    kpi_7  = (orders_df.loc[orders_df["CreatedDate"] >= last7, "ADC_Status__c"]
              .value_counts().rename_axis("Status").reset_index(name="Count_7d"))
    status_kpis = (pd.merge(kpi_30, kpi_7, on="Status", how="outer")
                     .fillna(0)
                     .sort_values("Count_30d", ascending=False))

    # SLA (days) — CreatedDate → Order Approved Date (approved is a DATE)
    if orders_df["ADC_Order_Approved_Date__c"].notna().any():
        approved = orders_df.dropna(subset=["ADC_Order_Approved_Date__c"]).copy()
        # normalize both to midnight for days math
        approved["tt_days"] = (
            approved["ADC_Order_Approved_Date__c"].dt.normalize()
            - approved["CreatedDate"].dt.normalize()
        ).dt.days
        sla_summary = approved["tt_days"].describe(percentiles=[0.5, 0.9, 0.95]).to_frame("Days")
    else:
        sla_summary = pd.DataFrame()

    # Void reasons
    void_reasons = (orders_df["ADC_Reason__c"].value_counts(dropna=True)
                    .rename_axis("Void_Reason").reset_index(name="Count"))

    # Aging of open orders (adjust statuses to your org’s picklist)
    open_statuses = ["Submitted", "Pending Approval", "In Progress"]
    open_df = orders_df[orders_df["ADC_Status__c"].isin(open_statuses)].copy()
    if not open_df.empty:
        open_df["age_hours"] = (now - open_df["CreatedDate"]).dt.total_seconds() / 3600.0
        bins   = [-1, 4, 24, 72, 168, 1e9]
        labels = ["<=4h", "4–24h", "24–72h", "3–7d", ">7d"]
        aging = (pd.cut(open_df["age_hours"], bins=bins, labels=labels)
                 .value_counts().reindex(labels).fillna(0).astype(int).reset_index())
        aging.columns = ["Aging_Bucket", "Open_Count"]
    else:
        aging = pd.DataFrame({"Aging_Bucket": ["<=4h","4–24h","24–72h","3–7d",">7d"], "Open_Count": [0,0,0,0,0]})

    # ---- Save outputs ----
    out_dir = "carbon_reporting_out"
    os.makedirs(out_dir, exist_ok=True)
    status_kpis.to_csv(f"{out_dir}/status_kpis.csv", index=False)
    void_reasons.to_csv(f"{out_dir}/void_reasons.csv", index=False)
    aging.to_csv(f"{out_dir}/aging_open_orders.csv", index=False)
    if not sla_summary.empty:
        sla_summary.to_csv(f"{out_dir}/approval_sla_summary.csv")

    # ---- Simple digest ----
    def md_table(df, max_rows=10):
        if df.empty: return "_No data_"
        _df = df.head(max_rows)
        header = "| " + " | ".join(_df.columns) + " |\n"
        sep    = "| " + " | ".join(["---"]*len(_df.columns)) + " |\n"
        rows   = "\n".join("| " + " | ".join(str(v) for v in row) + " |" for row in _df.to_records(index=False))
        return header + sep + rows

    digest = f"""
# Carbon NCO Web Sampling – 30-Day Snapshot
**Generated (UTC):** {now.strftime('%Y-%m-%d %H:%M')}

## Status (30d & 7d)
{md_table(status_kpis)}

## Approval SLA (days; Approved only)
{md_table(sla_summary.reset_index().rename(columns={{'index':'Stat'}})) if not sla_summary.empty else "_No approved orders in window_"}

## Void Reasons (30d)
{md_table(void_reasons)}

## Open Order Aging (now)
{md_table(aging)}
"""
    with open(f"{out_dir}/daily_digest.md", "w", encoding="utf-8") as f:
        f.write(digest)
    print("Wrote outputs to:", out_dir)


Connected to: salesassist--full.sandbox.my.salesforce.com
Order types in last 30d: ['HCP Firsthand Experience', 'Management Orders-CGM', 'NCO Web Sampling']
Wrote outputs to: carbon_reporting_out


In [3]:
# ---- Plotly dashboards for business-facing monitoring ----
# pip install if needed:  pip install plotly

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# --- Safety: build the aggregates if they aren't already present ---
if "ADC_Order_Approved_Date__c" in orders_df.columns:
    orders_df["CreatedDate"] = pd.to_datetime(orders_df["CreatedDate"], utc=True, errors="coerce")
    orders_df["ADC_Order_Approved_Date__c"] = pd.to_datetime(orders_df["ADC_Order_Approved_Date__c"], errors="coerce")

# Order types
order_type_counts = (
    orders_df["ADC_Order_Type__c"]
    .value_counts(dropna=False)
    .rename_axis("Order_Type")
    .reset_index(name="Count")
)

# Statuses
status_counts = (
    orders_df["ADC_Status__c"]
    .value_counts(dropna=False)
    .rename_axis("Status")
    .reset_index(name="Count")
)

# Daily trend (UTC)
orders_df["day"] = orders_df["CreatedDate"].dt.tz_convert("UTC").dt.date
trend = orders_df.groupby("day").size().rename("Orders").reset_index()

# SLA (days) – CreatedDate → ApprovedDate (if available)
approved = orders_df.dropna(subset=["ADC_Order_Approved_Date__c"]).copy()
if not approved.empty:
    approved["tt_days"] = (
        approved["ADC_Order_Approved_Date__c"].dt.normalize()
        - approved["CreatedDate"].dt.normalize()
    ).dt.days

# Open order aging (tweak open statuses as needed)
open_statuses = ["Submitted", "Pending", "Pending Approval", "Draft", "In Progress"]
open_df = orders_df[orders_df["ADC_Status__c"].isin(open_statuses)].copy()
aging = pd.DataFrame({"Aging_Bucket": ["<=1d","1–3d","3–7d",">7d"], "Open_Count": [0,0,0,0]})
if not open_df.empty:
    hours = (pd.Timestamp.utcnow() - open_df["CreatedDate"]).dt.total_seconds() / 3600.0
    bins = [-1, 24, 72, 168, 1e9]
    labels = ["<=1d","1–3d","3–7d",">7d"]
    bucketed = pd.cut(hours, bins=bins, labels=labels)
    aging = (bucketed.value_counts().reindex(labels).fillna(0).astype(int)
             .rename_axis("Aging_Bucket").reset_index(name="Open_Count"))

# ---------- Plotly figures ----------
fig_type = px.bar(order_type_counts, x="Order_Type", y="Count",
                  title="Orders by Type (Last 30 Days)")
fig_type.update_layout(xaxis_title="Order Type", yaxis_title="Count")

fig_status = px.bar(status_counts, x="Status", y="Count",
                    title="Orders by Status (Last 30 Days)")
fig_status.update_layout(xaxis_title="Status", yaxis_title="Count")

fig_trend = px.line(trend, x="day", y="Orders", markers=True,
                    title="Daily Orders (UTC, Last 30 Days)")
fig_trend.update_layout(xaxis_title="Day", yaxis_title="Orders")

if not approved.empty:
    fig_sla = px.histogram(approved, x="tt_days", nbins=20,
                           title="Approval Time (Days) – Distribution")
    fig_sla.update_layout(xaxis_title="Days to Approval", yaxis_title="Orders")
else:
    fig_sla = go.Figure().add_annotation(text="No approved orders with dates in window",
                                         x=0.5, y=0.5, showarrow=False)
    fig_sla.update_layout(title="Approval Time (Days) – Distribution")

fig_aging = px.bar(aging, x="Aging_Bucket", y="Open_Count",
                   title="Open Order Aging (Based on CreatedDate → Now)")
fig_aging.update_layout(xaxis_title="Aging Bucket", yaxis_title="Open Orders")

# Show inline (notebook)
fig_type.show()
fig_status.show()
fig_trend.show()
fig_sla.show()
fig_aging.show()




In [ ]:
import os, pandas as pd, numpy as np
from simple_salesforce import Salesforce
from simple_salesforce.exceptions import SalesforceGeneralError

# --- connect (sandbox) ---
sf = Salesforce(
    instance_url="https://salesassist--full.sandbox.my.salesforce.com",
    # session_id="",
    version="61.0"
)
print("Connected to:", sf.sf_instance)

PARENT_OBJ = "ADC_Compliance_Order__c"
ORDER_TYPE_FIELD = "ADC_Order_Type__c"
STATUS_FIELD = "ADC_Status__c"
APPROVED_DATE_FIELD = "ADC_Order_Approved_Date__c"  # date (optional)

# -------- discover child relationship (Order Products) + quantity field --------
pdesc = sf.restful(f"sobjects/{PARENT_OBJ}/describe")
candidates = []
for cr in pdesc["childRelationships"]:
    # heuristics: relationship that looks like Order Product(s)
    child = cr["childSObject"]
    rel   = cr.get("relationshipName")
    lbl   = cr.get("field", "")
    if any(s in (child or "").lower() for s in ["order_product", "orderproduct", "adc_order_product"]) or \
       any(s in (rel or "").lower() for s in ["order_products", "orderproduct", "order_products__r"]):
        candidates.append((child, rel))

if not candidates:
    raise RuntimeError("Couldn't find an Order Product child relationship via describe(). Open the object in Setup and share the child API name.")

# pick the first plausible
CHILD_OBJ, REL_NAME = candidates[0]
cdesc = sf.restful(f"sobjects/{CHILD_OBJ}/describe")

# find a quantity-like field on the child object
qty_field = None
for f in cdesc["fields"]:
    nm, lbl = f["name"].lower(), (f.get("label") or "").lower()
    if "quantity" in nm or "quantity" in lbl:
        qty_field = f["name"]; break

if not qty_field:
    raise RuntimeError(f"Couldn't find a 'Quantity' field on {CHILD_OBJ}. Check child object fields in Setup/Salesforce Inspector.")

print(f"Child relationship: {REL_NAME} on {CHILD_OBJ}; Quantity field: {qty_field}")

# -------- run parent + child (LAST_N_DAYS:30) --------
soql = f"""
SELECT Id, Name, {ORDER_TYPE_FIELD}, {STATUS_FIELD}, CreatedDate, {APPROVED_DATE_FIELD},
       (SELECT Id, Name, {qty_field} FROM {REL_NAME})
FROM {PARENT_OBJ}
WHERE CreatedDate = LAST_N_DAYS:30
"""
try:
    res = sf.query_all(soql)
except SalesforceGeneralError as e:
    raise RuntimeError(f"SOQL failed: {e.content}") from e

parents = res.get("records", [])
for r in parents: r.pop("attributes", None)
orders_df = pd.json_normalize(parents)

# explode the child rows (Order Products)
if f"{REL_NAME}.records" in orders_df.columns:
    exploded = orders_df.explode(f"{REL_NAME}.records", ignore_index=True)
    # drop parents w/o child rows
    exploded = exploded[exploded[f"{REL_NAME}.records"].notna()].copy()
    # flatten child record dict
    child_norm = pd.json_normalize(exploded[f"{REL_NAME}.records"])
    # merge back
    child_df = pd.concat([exploded.drop(columns=[f"{REL_NAME}.records"]), child_norm.add_prefix("child.")], axis=1)
else:
    child_df = pd.DataFrame()

# cast dates
if "CreatedDate" in orders_df:
    orders_df["CreatedDate"] = pd.to_datetime(orders_df["CreatedDate"], utc=True, errors="coerce")
if APPROVED_DATE_FIELD in orders_df:
    orders_df[APPROVED_DATE_FIELD] = pd.to_datetime(orders_df[APPROVED_DATE_FIELD], errors="coerce")

# ---------------- KPIs ----------------
# 1) Counts by Order Type
order_type_counts = (orders_df[ORDER_TYPE_FIELD]
                     .value_counts()
                     .rename_axis("Order_Type")
                     .reset_index(name="Order_Count"))

# 2) Counts by Status
status_counts = (orders_df[STATUS_FIELD]
                 .value_counts()
                 .rename_axis("Status")
                 .reset_index(name="Order_Count"))

# 3) Total samples (quantity) by Order Type (from child rows)
if not child_df.empty and f"child.{qty_field}" in child_df:
    # coerce qty to numeric
    child_df[f"child.{qty_field}"] = pd.to_numeric(child_df[f"child.{qty_field}"], errors="coerce").fillna(0)
    qty_by_type = (child_df
                   .groupby(ORDER_TYPE_FIELD, dropna=False)[f"child.{qty_field}"]
                   .sum()
                   .reset_index()
                   .rename(columns={f"child.{qty_field}": "Total_Samples"}))
else:
    qty_by_type = pd.DataFrame(columns=[ORDER_TYPE_FIELD, "Total_Samples"])

# 4) Focus: NCO Web Sampling (orders + samples)
NCO_LABEL = "NCO Web Sampling"
nco_orders = int(order_type_counts.loc[order_type_counts["Order_Type"]==NCO_LABEL, "Order_Count"].sum()) if not order_type_counts.empty else 0
nco_samples = int(qty_by_type.loc[qty_by_type[ORDER_TYPE_FIELD]==NCO_LABEL, "Total_Samples"].sum()) if not qty_by_type.empty else 0
total_orders = int(len(orders_df))
nco_pct = (nco_orders / total_orders * 100) if total_orders else 0.0

# 5) SLA (days) if approved date present
if APPROVED_DATE_FIELD in orders_df and orders_df[APPROVED_DATE_FIELD].notna().any():
    tmp = orders_df.dropna(subset=[APPROVED_DATE_FIELD]).copy()
    tmp["tt_days"] = (tmp[APPROVED_DATE_FIELD].dt.normalize() - tmp["CreatedDate"].dt.normalize()).dt.days
    sla = tmp["tt_days"].describe(percentiles=[0.5,0.9,0.95]).to_frame("Days")
else:
    sla = pd.DataFrame()

# 6) Open aging buckets
open_statuses = ["Submitted","Pending","Pending Approval","Draft","In Progress"]
now = pd.Timestamp.utcnow()
open_df = orders_df[orders_df[STATUS_FIELD].isin(open_statuses)].copy()
if not open_df.empty:
    open_df["age_hours"] = (now - pd.to_datetime(open_df["CreatedDate"], utc=True)).dt.total_seconds()/3600
    bins = [-1,24,72,168,1e9]; labels = ["<=1d","1–3d","3–7d",">7d"]
    aging = (pd.cut(open_df["age_hours"], bins=bins, labels=labels)
             .value_counts().reindex(labels).fillna(0).astype(int).reset_index())
    aging.columns = ["Aging_Bucket","Open_Count"]
else:
    aging = pd.DataFrame({"Aging_Bucket":["<=1d","1–3d","3–7d",">7d"],"Open_Count":[0,0,0,0]})

# ---------- Print a compact business digest ----------
print("\n=== Compliance Orders – 30 Day Summary ===")
print(f"Total Orders: {total_orders}")
print("By Order Type:\n", order_type_counts.to_string(index=False))
print("\nBy Status:\n", status_counts.to_string(index=False))
if not qty_by_type.empty:
    print("\nTotal Samples (Qty) by Order Type:\n", qty_by_type.rename(columns={ORDER_TYPE_FIELD:"Order_Type"}).to_string(index=False))
else:
    print("\nTotal Samples (Qty) by Order Type: (no child rows/quantity field found)")
print(f"\nNCO Web Sampling — Orders: {nco_orders} ({nco_pct:.1f}%), Samples: {nco_samples}")

if not sla.empty:
    print("\nSLA (days, Created → Approved):")
    print(sla.to_string())
else:
    print("\nSLA: no approved-date data available in window.")

print("\nOpen Order Aging (counts):\n", aging.to_string(index=False))


Connected to: salesassist--full.sandbox.my.salesforce.com
Child relationship: Order_Products__r on ADC_Order_Product__c; Quantity field: ADC_Quantity__c

=== Compliance Orders – 30 Day Summary ===
Total Orders: 216
By Order Type:
               Order_Type  Order_Count
        NCO Web Sampling          207
   Management Orders-CGM            8
HCP Firsthand Experience            1

By Status:
                         Status  Order_Count
Closed - Ready for fulfillment           64
                     Submitted           62
               Completed Order           50
                         Draft           27
                        Cancel            7
                       Pending            4
                      Approved            1
                      Returned            1

Total Samples (Qty) by Order Type:
               Order_Type  Total_Samples
HCP Firsthand Experience            9.0
   Management Orders-CGM           22.0
        NCO Web Sampling         1939.0
           

In [5]:
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# ---------- 1) Orders by Type (bar + pie) ----------
fig_type_bar = px.bar(
    order_type_counts,
    x="Order_Type", y="Order_Count",
    title="Orders by Type (Last 30 Days)",
)
fig_type_pie = px.pie(
    order_type_counts,
    names="Order_Type", values="Order_Count",
    title="Order Mix by Type (Last 30 Days)",
    hole=0.45
)

# ---------- 2) Orders by Status (bar) ----------
fig_status = px.bar(
    status_counts,
    x="Status", y="Order_Count",
    title="Orders by Status (Last 30 Days)",
)
fig_status.update_layout(xaxis_tickangle=-30)

# ---------- 3) Samples by Order Type (bar) ----------
if not qty_by_type.empty:
    fig_samples = px.bar(
        qty_by_type.rename(columns={ "ADC_Order_Type__c": "Order_Type" }),
        x="Order_Type", y="Total_Samples",
        title="Total Samples (Qty) by Order Type (Last 30 Days)",
        text="Total_Samples"
    )
    fig_samples.update_traces(texttemplate="%{text:.0f}", textposition="outside")
else:
    fig_samples = go.Figure().add_annotation(
        text="No order-product quantities found for this window.",
        x=0.5, y=0.5, showarrow=False
    )
    fig_samples.update_layout(title="Total Samples (Qty) by Order Type")

# ---------- 4) NCO Web Sampling focus (indicator) ----------
NCO_LABEL = "NCO Web Sampling"
total_orders = int(order_type_counts["Order_Count"].sum())
nco_orders = int(order_type_counts.loc[order_type_counts["Order_Type"]==NCO_LABEL, "Order_Count"].sum())
nco_pct = (nco_orders / total_orders * 100) if total_orders else 0.0

nco_samples = 0
if not qty_by_type.empty:
    nco_samples = int(qty_by_type.loc[qty_by_type["ADC_Order_Type__c"]==NCO_LABEL, "Total_Samples"].sum())

fig_nco = make_fig = go.Figure()
fig_nco.add_trace(go.Indicator(
    mode="number+delta",
    title={"text": f"NCO Web Sampling – Orders"},
    value=nco_orders,
    delta={"reference": total_orders, "relative": True, "valueformat": ".1%"}
))
fig_nco.add_trace(go.Indicator(
    mode="number",
    title={"text": "NCO Web Sampling – Samples"},
    value=nco_samples,
    domain={"x":[0.55,1.0], "y":[0,1]}
))
fig_nco.update_layout(title=f"NCO Focus: {nco_orders} orders ({nco_pct:.1f}%) · {nco_samples} samples")

# ---------- 5) Daily trend (total) ----------
orders_df["CreatedDate"] = pd.to_datetime(orders_df["CreatedDate"], utc=True, errors="coerce")
orders_df["day"] = orders_df["CreatedDate"].dt.tz_convert("UTC").dt.date
trend = orders_df.groupby("day").size().rename("Orders").reset_index()
fig_trend = px.line(trend, x="day", y="Orders", markers=True, title="Daily Orders (UTC, Last 30 Days)")

# ---------- 6) Daily stacked by Status (great for ops) ----------
stack = (orders_df
         .groupby(["day","ADC_Status__c"])
         .size().rename("Orders").reset_index())
fig_stack = px.area(
    stack.sort_values("day"), x="day", y="Orders", color="ADC_Status__c",
    title="Daily Orders by Status (Stacked, Last 30 Days)",
    groupnorm=None
)

# ---------- 7) Approval SLA (days) distribution ----------
if "approved" in locals() and not approved.empty and "tt_days" in approved.columns:
    fig_sla = px.histogram(
        approved, x="tt_days", nbins=20,
        title="Approval Time Distribution (Days)"
    )
    fig_sla.update_layout(xaxis_title="Days to Approval", yaxis_title="Orders")
else:
    fig_sla = go.Figure().add_annotation(
        text="No approved-date data available for SLA.",
        x=0.5, y=0.5, showarrow=False
    )
    fig_sla.update_layout(title="Approval Time Distribution (Days)")

# ---------- 8) Open Order Aging ----------
fig_aging = px.bar(
    aging, x="Aging_Bucket", y="Open_Count",
    title="Open Order Aging (Created → Now)",
    text="Open_Count"
)
fig_aging.update_traces(texttemplate="%{text}", textposition="outside")

# ---------- Show inline ----------
for f in [fig_type_bar, fig_type_pie, fig_status, fig_samples, fig_trend, fig_stack, fig_sla, fig_aging, fig_nco]:
    f.show()

# ---------- Optional: export a single HTML you can share ----------
os.makedirs("carbon_reporting_out", exist_ok=True)
html_path = os.path.join("carbon_reporting_out", "compliance_orders_report.html")

sections = [
    ("Orders by Type (Bar)", fig_type_bar),
    ("Orders by Type (Pie)", fig_type_pie),
    ("Orders by Status", fig_status),
    ("Samples by Order Type", fig_samples),
    ("Daily Orders", fig_trend),
    ("Daily Orders by Status (Stacked)", fig_stack),
    ("Approval Time (Days)", fig_sla),
    ("Open Order Aging", fig_aging),
    ("NCO Focus", fig_nco),
]

html = ["<html><head><meta charset='utf-8'><title>Compliance Orders – 30-Day Report</title></head><body style='font-family:Inter,system-ui,sans-serif'>"]
html.append(f"<h1>Compliance Orders – Last 30 Days</h1><p>Total Orders: {total_orders} · NCO Orders: {nco_orders} ({nco_pct:.1f}%) · NCO Samples: {nco_samples}</p>")
for title, fig in sections:
    html.append(f"<h2>{title}</h2>")
    html.append(fig.to_html(full_html=False, include_plotlyjs='cdn'))
html.append("</body></html>")

with open(html_path, "w", encoding="utf-8") as f:
    f.write("\n".join(html))

print("Wrote:", html_path)


Wrote: carbon_reporting_out/compliance_orders_report.html


In [6]:
# ===== NCO Web Sampling – focused charts =====
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

NCO_LABEL = "NCO Web Sampling"
ORDER_TYPE_FIELD = "ADC_Order_Type__c"
STATUS_FIELD = "ADC_Status__c"

# 1) Filter parents to NCO
nco_df = orders_df.loc[orders_df[ORDER_TYPE_FIELD] == NCO_LABEL].copy()
nco_df["CreatedDate"] = pd.to_datetime(nco_df["CreatedDate"], utc=True, errors="coerce")
nco_df["day"] = nco_df["CreatedDate"].dt.tz_convert("UTC").dt.date

# 2) Orders by Status (NCO only)
nco_status_counts = (nco_df[STATUS_FIELD]
                     .value_counts()
                     .rename_axis("Status")
                     .reset_index(name="Orders"))
fig_nco_status = px.bar(
    nco_status_counts, x="Status", y="Orders",
    title="NCO Web Sampling – Orders by Status (Last 30 Days)"
)
fig_nco_status.update_layout(xaxis_tickangle=-30)
fig_nco_status.show()

# 3) Daily Orders (NCO only)
nco_trend = nco_df.groupby("day").size().rename("Orders").reset_index()
fig_nco_trend = px.line(
    nco_trend, x="day", y="Orders", markers=True,
    title="NCO Web Sampling – Daily Orders (UTC, Last 30 Days)"
)
fig_nco_trend.update_layout(xaxis_title="Day", yaxis_title="Orders")
fig_nco_trend.show()

# 4) Daily Orders by Status (stacked area; NCO only)
nco_stack = (nco_df.groupby(["day", STATUS_FIELD])
             .size().rename("Orders").reset_index())
fig_nco_stack = px.area(
    nco_stack.sort_values("day"),
    x="day", y="Orders", color=STATUS_FIELD,
    title="NCO Web Sampling – Daily Orders by Status (Stacked, Last 30 Days)"
)
fig_nco_stack.update_layout(xaxis_title="Day", yaxis_title="Orders")
fig_nco_stack.show()

# 5) Samples: quantities from child Order Products, NCO only
# (child_df was built earlier by exploding the child rel; keep its field names)
#   - child quantity field was auto-detected → qty_field
#   - child columns are prefixed with "child."
if "child.Id" in child_df.columns:
    nco_child = child_df.loc[child_df[ORDER_TYPE_FIELD] == NCO_LABEL].copy()
    # coerce qty to numeric
    nco_child[f"child.{qty_field}"] = pd.to_numeric(nco_child[f"child.{qty_field}"], errors="coerce").fillna(0)

    # 5a) Total samples by product (top 15)
    prod_col = "child.Name" if "child.Name" in nco_child.columns else "child.ProductCode" if "child.ProductCode" in nco_child.columns else "child.Id"
    samples_by_prod = (nco_child.groupby(prod_col)[f"child.{qty_field}"]
                       .sum().reset_index().rename(columns={prod_col: "Product", f"child.{qty_field}": "Total_Samples"})
                       .sort_values("Total_Samples", ascending=False).head(15))
    fig_nco_samples_by_prod = px.bar(
        samples_by_prod, x="Product", y="Total_Samples",
        title="NCO Web Sampling – Samples by Product (Top 15, Last 30 Days)", text="Total_Samples"
    )
    fig_nco_samples_by_prod.update_traces(texttemplate="%{text:.0f}", textposition="outside")
    fig_nco_samples_by_prod.update_layout(xaxis_tickangle=-30)
    fig_nco_samples_by_prod.show()

    # 5b) Daily samples (quantity) – sum over products
    # Use parent CreatedDate as "order date" driver
    nco_child["CreatedDate"] = pd.to_datetime(nco_child["CreatedDate"], utc=True, errors="coerce")
    nco_child["day"] = nco_child["CreatedDate"].dt.tz_convert("UTC").dt.date
    samples_trend = (nco_child.groupby("day")[f"child.{qty_field}"]
                     .sum().reset_index().rename(columns={f"child.{qty_field}": "Samples"}))
    fig_nco_samples_trend = px.line(
        samples_trend, x="day", y="Samples", markers=True,
        title="NCO Web Sampling – Daily Samples (Qty, Last 30 Days)"
    )
    fig_nco_samples_trend.update_layout(xaxis_title="Day", yaxis_title="Samples")
    fig_nco_samples_trend.show()
else:
    print("No child order-product rows available; skipping product/sample charts.")

# 6) Optional: one HTML file with these NCO-only visuals
import os
os.makedirs("carbon_reporting_out", exist_ok=True)
html_path = os.path.join("carbon_reporting_out", "nco_web_sampling_only.html")
sections = [
    ("NCO Orders by Status", fig_nco_status),
    ("NCO Daily Orders", fig_nco_trend),
    ("NCO Daily Orders by Status (Stacked)", fig_nco_stack),
]
if 'fig_nco_samples_by_prod' in locals():
    sections.append(("NCO Samples by Product (Top 15)", fig_nco_samples_by_prod))
if 'fig_nco_samples_trend' in locals():
    sections.append(("NCO Daily Samples (Qty)", fig_nco_samples_trend))

html = ["<html><head><meta charset='utf-8'><title>NCO Web Sampling – 30-Day Report</title></head><body style='font-family:Inter,system-ui,sans-serif'>"]
html.append(f"<h1>NCO Web Sampling – Last 30 Days</h1><p>Total NCO Orders: {len(nco_df)}</p>")
for title, fig in sections:
    html.append(f"<h2>{title}</h2>")
    html.append(fig.to_html(full_html=False, include_plotlyjs='cdn'))
html.append("</body></html>")
with open(html_path, "w", encoding="utf-8") as f:
    f.write("\n".join(html))
print("Wrote:", html_path)


Wrote: carbon_reporting_out/nco_web_sampling_only.html


In [7]:
# ========= ONE-CELL PIPELINE: Query -> Flatten -> Inspect Channels -> NCO HCP Charts =========
import os, pandas as pd
import plotly.express as px
from simple_salesforce import Salesforce, SalesforceGeneralError

# ---- CONFIG ----
PARENT_OBJ       = "ADC_Compliance_Order__c"
ORDER_TYPE_FIELD = "ADC_Order_Type__c"
STATUS_FIELD     = "ADC_Status__c"
CHANNEL_FIELD    = "ADC_Channel__c"
HCP_NAME_FIELD   = "ADC_HCP__c"
HCP_ID_FIELD     = "ADC_HCP_ID__c"
NCO_LABEL        = "NCO Web Sampling"
DAYS             = 60                  # widen window if needed (was 30)
CHANNEL_FILTER   = None                # e.g., ["Web"] or ["WEB","Digital"]; None = no filter
TOPN             = 20
OUT_DIR          = "carbon_reporting_out"

# (Assumes you already have sf = Salesforce(...))
# sf = Salesforce(instance_url=..., session_id=..., version="61.0")

# ---------- 1) Discover child relationship + quantity field ----------
pdesc = sf.restful(f"sobjects/{PARENT_OBJ}/describe")
REL_NAME, CHILD_OBJ, QTY_FIELD = None, None, None

for cr in pdesc["childRelationships"]:
    rel = cr.get("relationshipName")
    child_sobj = cr.get("childSObject")
    if not rel or not child_sobj:
        continue
    try:
        cdesc = sf.restful(f"sobjects/{child_sobj}/describe")
    except Exception:
        continue
    q_candidate = None
    for f in cdesc["fields"]:
        nm = f.get("name","").lower()
        lb = f.get("label","").lower()
        if ("quantity" in nm or "quantity" in lb) and f.get("type") in {"double","int","currency","percent","long","number"}:
            q_candidate = f["name"]; break
    if q_candidate:
        REL_NAME, CHILD_OBJ, QTY_FIELD = rel, child_sobj, q_candidate
        break

if not REL_NAME or not QTY_FIELD:
    raise RuntimeError("Couldn't auto-detect the child relationship/quantity field.")

print(f"Using child relationship: {REL_NAME} (object: {CHILD_OBJ}), quantity field: {QTY_FIELD}")

# ---------- 2) Build ONE SOQL (no channel filter yet) ----------
soql = f"""
SELECT Id, Name, CreatedDate,
       {STATUS_FIELD}, {ORDER_TYPE_FIELD}, {CHANNEL_FIELD},
       {HCP_NAME_FIELD}, {HCP_ID_FIELD},
       (SELECT Id, Name, {QTY_FIELD} FROM {REL_NAME})
FROM {PARENT_OBJ}
WHERE CreatedDate = LAST_N_DAYS:{DAYS}
"""
res = sf.query_all(soql)
parents = res.get("records", [])
for r in parents:
    r.pop("attributes", None)

if not parents:
    raise RuntimeError(f"No orders returned for LAST_N_DAYS:{DAYS} (without channel filter). Try increasing DAYS.")

# ---------- 3) Flatten parent + child ----------
def child_key_for_record(rec):
    for k, v in rec.items():
        if isinstance(v, dict) and "records" in v and isinstance(v["records"], list):
            return k
    return None

child_key = None
for p in parents:
    child_key = child_key_for_record(p)
    if child_key: break

flat_rows = []
for p in parents:
    base = {k: v for k, v in p.items() if k != child_key}
    children = (p.get(child_key, {}) or {}).get("records", [])
    if not children:
        flat_rows.append(base)
    else:
        for ch in children:
            ch = dict(ch); ch.pop("attributes", None)
            flat_rows.append({**base, **{f"child.{k}": v for k, v in ch.items()}})

flat_df = pd.DataFrame(flat_rows)
orders_df = pd.DataFrame([{k: v for k, v in p.items() if k != child_key} for p in parents])
child_cols = [c for c in flat_df.columns if c.startswith("child.")]
child_df = flat_df.dropna(subset=child_cols, how="all").copy()

qty_col = f"child.{QTY_FIELD}"
child_df[qty_col] = pd.to_numeric(child_df[qty_col], errors="coerce").fillna(0)

# ---------- 4) Inspect channels & optionally filter ----------
channels_found = sorted(orders_df[CHANNEL_FIELD].dropna().astype(str).unique())
print("Distinct channels in last", DAYS, "days:", channels_found)

if CHANNEL_FILTER:
    filt = orders_df[CHANNEL_FIELD].astype(str).isin(CHANNEL_FILTER)
    keep_ids = set(orders_df.loc[filt, "Id"])
    if not keep_ids:
        raise RuntimeError(f"No orders matched CHANNEL_FILTER={CHANNEL_FILTER}. "
                           f"Available values: {channels_found}")
    child_df = child_df[child_df["Id"].isin(keep_ids)].copy()
    print(f"Applied channel filter {CHANNEL_FILTER}: kept {len(keep_ids)} parent orders.")
else:
    print("No channel filter applied (using all channels). "
          "Set CHANNEL_FILTER=['Web'] (or the exact value you see above) if needed.")

# ---------- 5) NCO-only (Order Type) ----------
if ORDER_TYPE_FIELD not in child_df.columns:
    # join down essentials (should already be present, but safe to ensure)
    child_df = child_df.merge(
        orders_df[["Id", ORDER_TYPE_FIELD, STATUS_FIELD, HCP_NAME_FIELD, HCP_ID_FIELD]],
        on="Id", how="left"
    )

nco_child = child_df.loc[child_df[ORDER_TYPE_FIELD] == NCO_LABEL].copy()
if nco_child.empty:
    raise RuntimeError(f"No '{NCO_LABEL}' child rows found after filters. "
                       f"Try removing CHANNEL_FILTER or widening DAYS.")

# HCP display
def hcp_display(row):
    name = (row.get(HCP_NAME_FIELD) or "").strip()
    hid  = (row.get(HCP_ID_FIELD) or "").strip()
    return name if name else (hid if hid else "Unknown")
nco_child["HCP_Display"] = nco_child.apply(hcp_display, axis=1)

# ---------- 6) Aggregations ----------
hcp_samples = (nco_child.groupby("HCP_Display")[qty_col]
               .sum().reset_index()
               .rename(columns={qty_col:"Total_Samples"})
               .sort_values("Total_Samples", ascending=False))

hcp_orders = (nco_child.drop_duplicates(subset=["HCP_Display","Id"])
              .groupby("HCP_Display")["Id"]
              .nunique().reset_index()
              .rename(columns={"Id":"Order_Count"})
              .sort_values("Order_Count", ascending=False))

hcp_both = (hcp_samples.merge(hcp_orders, on="HCP_Display", how="outer")
            .fillna(0)
            .sort_values(["Total_Samples","Order_Count"], ascending=False))
total_samples_all = max(float(hcp_both["Total_Samples"].sum()), 1.0)
hcp_both["Rank"] = range(1, len(hcp_both) + 1)
hcp_both["Cum_Samples_%"] = (hcp_both["Total_Samples"].cumsum() / total_samples_all * 100).round(1)

# ---------- 7) Charts ----------
TOPN = min(TOPN, len(hcp_both))
top_samples = hcp_both.head(TOPN).copy()
top_orders  = hcp_both.sort_values("Order_Count", ascending=False).head(TOPN).copy()

fig_hcp_samples = px.bar(
    top_samples.sort_values("Total_Samples"),
    x="Total_Samples", y="HCP_Display",
    orientation="h",
    text="Total_Samples",
    labels={"Total_Samples":"Samples (Qty)","HCP_Display":"HCP"},
    title=f"{NCO_LABEL} — Top {TOPN} HCPs by Samples (Qty, Last {DAYS} Days)"
)
fig_hcp_samples.update_traces(texttemplate="%{text:.0f}", textposition="outside")
fig_hcp_samples.update_layout(margin=dict(l=220, r=40, t=60, b=40))
fig_hcp_samples.show()

fig_hcp_orders = px.bar(
    top_orders.sort_values("Order_Count"),
    x="Order_Count", y="HCP_Display",
    orientation="h",
    text="Order_Count",
    labels={"Order_Count":"Orders","HCP_Display":"HCP"},
    title=f"{NCO_LABEL} — Top {TOPN} HCPs by Orders (Last {DAYS} Days)"
)
fig_hcp_orders.update_traces(textposition="outside")
fig_hcp_orders.update_layout(margin=dict(l=220, r=40, t=60, b=40))
fig_hcp_orders.show()

mix_df = (nco_child[nco_child["HCP_Display"].isin(set(top_samples["HCP_Display"]))]
          .drop_duplicates(subset=["HCP_Display","Id"])
          .groupby(["HCP_Display", STATUS_FIELD])["Id"]
          .nunique().reset_index()
          .rename(columns={"Id":"Orders"}))
fig_status_mix = px.bar(
    mix_df, x="HCP_Display", y="Orders", color=STATUS_FIELD, barmode="stack",
    title=f"{NCO_LABEL} — Order Status Mix for Top {TOPN} HCPs"
)
fig_status_mix.update_layout(xaxis_tickangle=-25, margin=dict(t=60))
fig_status_mix.show()

# ---------- 8) Optional exports ----------
os.makedirs(OUT_DIR, exist_ok=True)
hcp_both.to_csv(os.path.join(OUT_DIR, "nco_hcp_samples_and_orders.csv"), index=False)
mix_df.to_csv(os.path.join(OUT_DIR, "nco_hcp_status_mix.csv"), index=False)
print("Saved CSVs to:", OUT_DIR)


Using child relationship: Order_Products__r (object: ADC_Order_Product__c), quantity field: ADC_Quantity__c
Distinct channels in last 60 days: ['HCP', 'Market Access', 'POC']
No channel filter applied (using all channels). Set CHANNEL_FILTER=['Web'] (or the exact value you see above) if needed.


Saved CSVs to: carbon_reporting_out
